# AERONAUTICAL PROJECT
The following project is based on the realization of virtual reality software for the aeronautical sector with the purpose of improving the learning experience for flight assistants, the goal is to include artificial intelligence and implement with the metahumans who will be inserted into the software.
for start the project we will begin making a review of some contents. 
## Interaction with virtual characters
One of the main ideas of this software is the "interaction with virtual characters" or "communication with virtual entities." In this scenario, "metahumanos" could be virtual characters, avatars or digital entities that can interact and communicate with users in a similar way to how humans interact with each other.
So to develop the idea we need tools for the creation of chatbots and virtual agents with advanced natural language processing capabilities.

In [ ]:
!pip install nltk

## First thing we do is import all the libraries 
we need and download nltk.
1. nltk (NATURAL LENGUAGE TOOLKIT) : used for natural language processing.
2. Wordnetlemmatizer: A lematizer is used to reduce words to its base or motto, which can be useful in text processing.
3. Lemmatizer = Wordnetlemmatizer (): Create an instance of the lematizer
4. Json: Our archive is made with Json.
5. Pickle: Used for saving trained models
6. Keras: for the Neural network.

In [1]:
import nltk
nltk.download()
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
from tensorflow.keras.optimizers import SGD
import random

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


Let's  3 create empty lists.
1. Words: stores the words we will process.
2. Classes: will store the classes.
3. Documments: will be used to save text documents.

In [2]:
words=[]
classes = []
documents = []

We open our Json archive.

In [3]:
with open("intents.json", "r") as archivo_json:
    datos_json = json.load(archivo_json)

let's check if it is working correctly

In [4]:
print(datos_json) #it is working fine but we need to keep cleanning the data

{'intents': [{'intent': 'Greetings', 'text': ['Bienvenida a bordo', 'Disculpe', 'Gracias por volar con nosotros'], 'response': ['Disculpe, Â¿podria ayudarme a abrochar mi cinturon de seguridad, por favor?', 'Perdon por molestar, Â¿podria brindarme asistencia para abrochar mi cinturon de seguridad, por favor?', 'Necesito un poco de ayuda con mi cinturon, Â¿puede asistirme?', 'Â¿SerÃ\xada tan amable de abrochar mi cinturon de seguridad?', 'Necesito ayuda con el cinturon de seguridad, Â¿podria, por favor?', 'Â¿Podria ofrecerme asistencia con el cinturon?', 'Disculpe, Â¿puede ocuparse de mi cinturon, por favor?', 'Por favor, Â¿me podria dar una mano con el cinturon?', 'Â¿Seria tan amable de abrochar mi cinturon de seguridad?', 'Necesito asistencia para abrochar mi cinturon, Â¿podria?', 'Â¿PodrÃ\xada brindarme ayuda con el cinturon de seguridad, por favor?', 'Disculpe, Â¿me podrÃ\xada dar una mano para abrochar el cinturon?', 'Â¿PodrÃ\xada ocuparse de mi cinturon de seguridad, por favor?', 

In [5]:
for intent in datos_json['intents']:
    for text in intent['text']:
        w = nltk.word_tokenize(text)
        words.extend(w)

We open the document list and add the tokenization, intent is the label if the name of the intent is not on the class list, it is added to this list. This ensures that Classses will contain all the unique labels of the intentions present in the data.

In [24]:
documents.append((w, intent['intent']))
if intent['intent'] not in classes:
    classes.append(intent['intent'])

We lemmatize W.lower is to put them in lowercase, Sorted is to hate alphabically for not repeat the same word.

In [7]:
words = [lemmatizer.lemmatize(w.lower())
for w in words]
words = sorted(list(set(words)))

In [8]:
classes = sorted(list(set(classes)))

In [9]:
print(len(documents), "documents")

1 documents


In [10]:
print(len(classes), "classes", classes)

1 classes ['Greetings']


We print the lemmatized words also we safe with pickles the information of the words and the classes

In [11]:
print(len(words), "unique lemmatized words", words)
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

30 unique lemmatized words ['a', 'ayudarle', 'bienvenida', 'bordo', 'buen', 'buenas', 'buenos', 'claro', 'con', 'de', 'deme', 'dia', 'disculpe', 'gracias', 'inmediato', 'momento', 'noches', 'nosotros', 'permitame', 'por', 'puedo', 'segundo', 'si', 'su', 'supuesto', 'tardes', 'tiquete', 'un', 'ver', 'volar']


In [12]:
training = []

In [13]:
output_empty = [0] * len(classes)

In [14]:
for doc in documents:
    bag = []
pattern_words = doc[0]

In [15]:
 pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]

We create a list binary list

In [16]:
bag.append(1) if w in pattern_words else bag.append(0)

This code is used to prepare training data for a text classification model

In [17]:
output_row = list(output_empty)
output_row[classes.index(doc[1])] = 1
training.append([bag, output_row])

In [18]:
random.shuffle(training)
training = np.array(training)

In [19]:
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


We create the Neural network

In [20]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [21]:
sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [22]:
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
print("model created")

Epoch 1/200


C:\Users\Luz Helena\anaconda3\Lib\site-packages\tensorflow\python\util\dispatch.py:1176: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


1/1 [==============================] - 1s 1s/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 2/200
1/1 [==============================] - 0s 16ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 3/200
1/1 [==============================] - 0s 12ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 4/200
1/1 [==============================] - 0s 18ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 5/200
1/1 [==============================] - 0s 20ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 6/200
1/1 [==============================] - 0s 12ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 7/200
1/1 [==============================] - 0s 18ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 8/200
1/1 [==============================] - 0s 18ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 9/200
1/1 [==============================] - 0s 17ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 10/200
1/1 [==============================] - 0s 16ms/step - loss: 0.0000e+00 - accuracy: 

1/1 [==============================] - 0s 13ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 81/200
1/1 [==============================] - 0s 15ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 82/200
1/1 [==============================] - 0s 20ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 83/200
1/1 [==============================] - 0s 18ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 84/200
1/1 [==============================] - 0s 14ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 85/200
1/1 [==============================] - 0s 17ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 86/200
1/1 [==============================] - 0s 12ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 87/200
1/1 [==============================] - 0s 17ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 88/200
1/1 [==============================] - 0s 24ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 89/200
1/1 [==============================] - 0s 9ms/step - loss: 0.0000e+00 - a

1/1 [==============================] - 0s 16ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 160/200
1/1 [==============================] - 0s 19ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 161/200
1/1 [==============================] - 0s 15ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 162/200
1/1 [==============================] - 0s 17ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 163/200
1/1 [==============================] - 0s 17ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 164/200
1/1 [==============================] - 0s 15ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 165/200
1/1 [==============================] - 0s 19ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 166/200
1/1 [==============================] - 0s 28ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 167/200
1/1 [==============================] - 0s 19ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 168/200
1/1 [==============================] - 0s 13ms/step - loss: 0.00

C:\Users\Luz Helena\anaconda3\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
